In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle (2).json to kaggle (2).json


{'kaggle (2).json': b'{"username":"mahajanbhushan","key":"88002547ce5a2790b7eda0352585b43c"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp "kaggle (2).json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d balraj98/deepglobe-road-extraction-dataset
!unzip -q deepglobe-road-extraction-dataset.zip -d dataset-folder

 99% 3.77G/3.79G [00:25<00:00, 175MB/s]
100% 3.79G/3.79G [00:25<00:00, 159MB/s]


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

def enhance_image(image):
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.5)
    image = image.convert('L')
    image_eq = cv2.equalizeHist(np.array(image))
    image_eq = Image.fromarray(image_eq)
    enhancer = ImageEnhance.Sharpness(image_eq)
    image_sharpened = enhancer.enhance(2.0)
    return image_sharpened

def load_images_and_masks(image_files, mask_files):
    images = []
    masks = []
    for img_path, mask_path in zip(image_files, mask_files):
        image = Image.open(img_path)
        image = enhance_image(image)
        image = image.resize((128, 128))
        images.append(np.array(image))
        mask = Image.open(mask_path)
        mask = mask.resize((128, 128))
        masks.append(np.array(mask))

    return images, masks
input_dir = "/content/dataset-folder/train"
image_files = sorted(glob(os.path.join(input_dir, '*sat.jpg')))
mask_files = sorted(glob(os.path.join(input_dir, '*mask.png')))
images, masks = load_images_and_masks(image_files, mask_files)

In [ ]:
# Convert to numpy arrays and normalize
images = np.array(images).astype('float32') / 255.0
masks = np.array(masks).astype('float32') / 255.0
masks = np.expand_dims(masks, axis=-1)

X_train, X_valid, y_train, y_valid = train_test_split(images, masks, test_size=0.1, random_state=42)

train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
valid_generator = valid_datagen.flow(X_valid, y_valid, batch_size=32)

In [ ]:
import numpy as np
images1 = np.array(images).astype('float32') / 255.0
masks1 = np.array(masks).astype('float32') / 255.0

if len(images1.shape) == 3:
    images1 = np.stack((images1,)*3, axis=-1)

if len(masks1.shape) == 3:
    masks1 = np.expand_dims(masks1, axis=-1)
X_train, X_valid, y_train, y_valid = train_test_split(images1, masks1, test_size=0.1, random_state=42)

train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
valid_generator = valid_datagen.flow(X_valid, y_valid, batch_size=32)

In [ ]:
from keras.applications import ResNet50, VGG16
from keras.layers import Input, Concatenate, Conv2D, UpSampling2D
from keras.models import Model
input_tensor = Input(shape=(128, 128, 3))
resnet_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
vgg_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
resnet_features = resnet_model(input_tensor)
vgg_features = vgg_model(input_tensor)
merged_features = Concatenate()([resnet_features, vgg_features])
x = Conv2D(64, (3, 3), activation='relu', padding='same')(merged_features)
output = UpSampling2D(size=(32, 32))(x)
output = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(output)

fusion_model = Model(inputs=input_tensor, outputs=output)
fusion_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
history = fusion_model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    validation_data=valid_generator,
    validation_steps=len(X_valid) // 32,
    epochs=10
)

58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
176/176 [==============================] - 98s 294ms/step - loss: 0.3798 - accuracy: 0.9171 - val_loss: 0.2821 - val_accuracy: 0.9187
Epoch 2/10
176/176 [==============================] - 49s 278ms/step - loss: 0.1596 - accuracy: 0.9199 - val_loss: 0.3570 - val_accuracy: 0.9187
Epoch 3/10
176/176 [==============================] - 49s 280ms/step - loss: 0.1543 - accuracy: 0.9199 - val_loss: 0.2313 - val_accuracy: 0.9187
Epoch 4/10
176/176 [==============================] - 49s 280ms/step - loss: 0.1503 - accuracy: 0.9199 - val_loss: 0.2912 - val_accuracy: 0.9187
Epoch 5/10
176/176 [==============================] - 49s 280ms/step - loss: 0.1479 - accuracy: 0.9199 - val_loss: 0.3027 - val_accuracy: 0.9187
Epoch 6/10
176/176 [==============================] - 49s 280ms/step - loss: 0.1469 - accuracy: 0.9199 - val_loss: 0.1633 - val_accuracy: 0.9187
Epoch 7/10
176/176 [==============================] - 49s 280ms/s

In [ ]:
import pickle
fusion_model_enhancement_file = "fusion_model_enhancement_file.pkl"
with open(fusion_model_enhancement_file, 'wb') as file:
    pickle.dump(fusion_model, file)

In [ ]:
import pickle
with open('/content/fusion_model_enhancment_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)
print("Training history saved!")

Training history saved!


In [2]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose
from tensorflow.keras.models import Model

def unet(input_size=(128, 128, 3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    conv1 = Dropout(0.1)(conv1)
    conv1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    conv2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = Dropout(0.1)(conv2)
    conv2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    conv3 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    conv4 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    conv5 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = Dropout(0.3)(conv5)
    conv5 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    # Decoder
    decoder6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
    decoder6 = concatenate([decoder6, conv4])
    conv6 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(decoder6)
    conv6 = Dropout(0.2)(conv6)
    conv6 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)
    decoder7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    decoder7 = concatenate([decoder7, conv3])
    conv7 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(decoder7)
    conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
    decoder8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    decoder8 = concatenate([decoder8, conv2])
    conv8 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(decoder8)
    conv8 = Dropout(0.1)(conv8)
    conv8 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv8)
    decoder9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    decoder9 = concatenate([decoder9, conv1], axis=3)
    conv9 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(decoder9)
    conv9 = Dropout(0.1)(conv9)
    conv9 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv9)
    # Output layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model
unet_model = unet()
unet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d_17 (Conv2D)          (None, 128, 128, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 dropout_9 (Dropout)         (None, 128, 128, 64)         0         ['conv2d_17[0][0]']           
                                                                                                  
 conv2d_18 (Conv2D)          (None, 128, 128, 64)         36928     ['dropout_9[0][0]']           
                                                                                              

In [ ]:
# Fit the model
history1 = unet_model.fit(
    train_generator,
    validation_data=valid_generator,
    steps_per_epoch=len(X_train) // 32,
    validation_steps=len(X_valid) // 32,
    epochs=10
)

Epoch 1/10
175/175 [==============================] - 98s 479ms/step - loss: 0.1943 - accuracy: 0.9152 - val_loss: 0.1670 - val_accuracy: 0.9192
Epoch 2/10
175/175 [==============================] - 81s 463ms/step - loss: 0.1532 - accuracy: 0.9199 - val_loss: 0.1432 - val_accuracy: 0.9185
Epoch 3/10
175/175 [==============================] - 81s 464ms/step - loss: 0.1362 - accuracy: 0.9197 - val_loss: 0.1380 - val_accuracy: 0.9171
Epoch 4/10
175/175 [==============================] - 81s 465ms/step - loss: 0.1237 - accuracy: 0.9199 - val_loss: 0.1167 - val_accuracy: 0.9195
Epoch 5/10
175/175 [==============================] - 81s 465ms/step - loss: 0.1163 - accuracy: 0.9203 - val_loss: 0.1116 - val_accuracy: 0.9195
Epoch 6/10
175/175 [==============================] - 81s 465ms/step - loss: 0.1103 - accuracy: 0.9210 - val_loss: 0.1076 - val_accuracy: 0.9200
Epoch 7/10
175/175 [==============================] - 81s 465ms/step - loss: 0.1054 - accuracy: 0.9217 - val_loss: 0.1053 - val_ac

In [ ]:
import pickle
unet_model_enhancement_file = "unet_model_enhancement_file.pkl"
with open(unet_model_enhancement_file, 'wb') as file:
    pickle.dump(unet_model, file)

In [ ]:
import pickle
with open('/content/unet_model_enhancment_history.pkl', 'wb') as file:
    pickle.dump(history1.history, file)
print("Training history saved!")

Training history saved!
